In [107]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
# application_df.head()

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME", 'STATUS','SPECIAL_CONSIDERATIONS','INCOME_AMT'], axis= 1)
# application_df.head()

# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than 1000?
replace_application = list(app_counts[app_counts <500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

class_counts = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than 3000?
replace_class = list(class_counts[class_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    

# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
# encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [111]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  len(X_train[0]) * 4
hidden_nodes_layer2 = len(X_train[0]) *3
hidden_nodes_layer3 = len(X_train[0]) *2
hidden_nodes_layer4 = len(X_train[0])
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# fifth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_131 (Dense)            (None, 124)               3968      
_________________________________________________________________
dense_132 (Dense)            (None, 93)                11625     
_________________________________________________________________
dense_133 (Dense)            (None, 62)                5828      
_________________________________________________________________
dense_134 (Dense)            (None, 1)                 63        
Total params: 21,484
Trainable params: 21,484
Non-trainable params: 0
_________________________________________________________________


In [112]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("optimize_4_checkpoints/",exist_ok=True)
checkpoint_path = "optimize_4_checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", 
            optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"), 
            metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 987us/step - loss: 0.6018 - accuracy: 0.6996
Epoch 2/100
804/804 [==============================] - 1s 985us/step - loss: 0.5596 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5662 - accuracy: 0.7249
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5644 - accuracy: 0.7245
Epoch 5/100
755/804 [===========================>..] - ETA: 0s - loss: 0.5619 - accuracy: 0.7266
Epoch 00005: saving model to optimize_4_checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5618 - accuracy: 0.7267
Epoch 6/100
804/804 [==============================] - 1s 952us/step - loss: 0.5604 - accuracy: 0.7271
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5578 - accuracy: 0.7282
Epoch 8/100
804/804 [==============================] - 1s 925us/step - loss: 0.5586 - accuracy: 0.7262
Epoch 9/100
804/804 [==========

In [110]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("optimize_4_trained_application.h5")

268/268 - 0s - loss: 0.5543 - accuracy: 0.7272
Loss: 0.5542847514152527, Accuracy: 0.7272303104400635
